In [1]:
import os
import json
import gzip
import torch
import pandas as pd
from urllib.request import urlopen
from tqdm import tqdm
import gensim.downloader as api
import numpy as np
from PIL import Image, UnidentifiedImageError
from torchvision import transforms
from sklearn.neighbors import NearestNeighbors

/home/a.tliamov/anaconda3/envs/my_env/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [1]:
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')
roberta.eval()  

NameError: name 'torch' is not defined

In [3]:
data_meta = []
with gzip.open('meta_Movies_and_TV.json.gz') as f:
    for l in tqdm(f):
        data_meta.append(json.loads(l.strip()))

df_meta = pd.DataFrame.from_dict(data_meta)
data = df_meta[["asin", "description" , "also_buy"]]
data.head()

data["description"] = data["description"].apply(lambda x: ". ".join(x))
data.head()

203766it [00:09, 21391.52it/s]
/home/a.tliamov/anaconda3/envs/my_env/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,asin,description,also_buy
0,0000695009,,[]
1,0000791156,,[]
2,0000143529,Disc 1: Flour Power (Scones; Shortcakes; South...,[]
3,0000143588,Barefoot Contessa Volume 2: On these three dis...,"[B002I5GNW4, B005WXPVMM, B009UY3W8O, B00N27ID1..."
4,0000143502,Rise and Swine (Good Eats Vol. 7) includes bon...,"[B000P1CKES, B000NR4CRM]"


In [4]:
data = data[data["description"].apply(len) > 0][["asin", "description"]]
data = data.drop_duplicates(subset=["asin", "description"])
data.count()

asin           156728
description    156728
dtype: int64

In [5]:

roberta.to("cuda")
roberta.eval()

def func(text):
    tokens = roberta.encode(text)
    last_layer_features = roberta.extract_features(tokens[:512])
    del tokens
    r = torch.mean(last_layer_features, axis=1)
    del last_layer_features
    return r.detach().cpu()
l = []
for text in tqdm(data["description"].to_list()):
    l.append(func(text))

# data["emb"] = data["description"].apply(lambda x: func(x))


100%|██████████| 156728/156728 [1:08:56<00:00, 37.89it/s]


In [6]:
l = [i.squeeze().numpy() for i in l]
l_g = np.array(l)
np.save("roberta_large.npy", l_g)

In [7]:
l2 = np.load("roberta_large.npy")

In [8]:

data["emb"] = list(l2)
asin_and_mean_emb = data.groupby("asin")['emb'].apply(np.mean)
asin_and_mean_emb = pd.DataFrame(asin_and_mean_emb)
asin_and_mean_emb.head()

,emb
asin,
0000143502,"[-0.027129566, 0.0033685567, -0.50562084, 0.05..."
0000143529,"[0.007927135, -0.004966187, -0.4968999, 0.0810..."
0000143561,"[0.017030109, 0.061291214, -0.34818393, 0.1359..."
0000143588,"[0.018717797, -0.05082816, -0.5001769, 0.10823..."
000073991X,"[0.09781578, 0.043600462, -0.38677564, -0.0952..."


In [17]:
class Trainer:
    def __init__(self, model):
        self.model = model
        
    def get_embeddings(self, images, preprocess=False):
        batch_size = 16
        image_arr = None
        for i in tqdm(range(0, len(images), batch_size)):
            # select batch of images
            batch = images[i:i+batch_size]
            # process and resize
            batch = processor(
                text=None,
                images=batch,
                return_tensors='pt',
                padding=True
            )['pixel_values'].to(device)
            # get image embeddings
            batch_emb = self.model.get_image_features(pixel_values=batch)
            # convert to numpy array
            batch_emb = batch_emb.squeeze(0)
            batch_emb = batch_emb.cpu().detach().numpy()
            # add to larger array of all image embeddings
            if image_arr is None:
                image_arr = batch_emb
            else:
                image_arr = np.concatenate((image_arr, batch_emb), axis=0)
        return image_arr

    
class Predictor:
    def __init__(self, asin_and_mean_emb):
        self.neigh = NearestNeighbors(n_neighbors=100, radius=0.4, metric="cosine")
        self.neigh.fit(asin_and_mean_emb["emb"].tolist())
        self.asin_and_mean_emb = asin_and_mean_emb
        all_meta_info = get_meta_data()
        self.also_buy_data = all_meta_info[all_meta_info["also_buy"].apply(len) > 0][["asin", "description", "also_buy"]]
        self.all_asins = self.asin_and_mean_emb.index.tolist()
        
    def make(self):
        asin_from_also_buy = set(self.also_buy_data["asin"].tolist())
        all_asins = self.asin_and_mean_emb.index.tolist()
        asin_with_emb = set(all_asins)
        correct_asins = list(asin_with_emb & asin_from_also_buy)

        
        emb_for_knn_predict = self.asin_and_mean_emb.loc[correct_asins]["emb"].tolist()
        items_for_knn_predict = self.asin_and_mean_emb.loc[correct_asins].index.tolist()
        
        K = 5
        predict_items = self.predict(emb_for_knn_predict, K)
        
        result = pd.DataFrame({"asin": items_for_knn_predict, "predict_items": predict_items})

        predict_and_real_items = self.also_buy_data.merge(result, on="asin")

        print(f"precision_at_{K-1}: {precision_at_k(predict_and_real_items, k=K-1)}")
        print(f"recall_at_{K-1}: {recall_at_k(predict_and_real_items, k=K-1)}")

            
    def predict(self, emb_for_knn_predict,k):
        predicted_item_pos = self.neigh.kneighbors(emb_for_knn_predict, k, return_distance=True)
        return list(np.array(self.all_asins)[predicted_item_pos[1].tolist()])
        

def precision_at_k(predict_and_real_items, k):
    return predict_and_real_items.apply(lambda x: len(list(set(x["also_buy"]) & set(x["predict_items"])))/k, axis=1).mean()

def recall_at_k(predict_and_real_items, k):
    return predict_and_real_items.apply(lambda x: len(list(set(x["also_buy"]) & set(x["predict_items"])))/len(set(x["also_buy"])), axis=1).mean()
    

def get_Image_PIL_list(files):
    images = []
    for i in files:
        input_batch = Image.open(i)
        if input_batch.mode == 'L':
            input_batch = input_batch.convert('RGB')
        images.append(input_batch)
    return images

def get_meta_data():
    data_meta = []
    with gzip.open('meta_Movies_and_TV.json.gz') as f:
        for l in tqdm(f):
            data_meta.append(json.loads(l.strip()))

    return pd.DataFrame.from_dict(data_meta)

In [18]:
predictor = Predictor(asin_and_mean_emb)
predictor.make()

203766it [00:10, 19754.12it/s]


precision_at_4: 0.09281142985460057
recall_at_4: 0.0316262832135125
